# Compute the Summer Days Climate Index for several years

Starting from the [**Summer Days workflow**](../Examples/Summer_Days_workflow.ipynb) example for the computation of the **Summer Days index** for one input NetCDF data set, the index can be computed for several years in parallel by exploiting the **iterative and parallel interfaces** made available by the Ophidia Workflow Management System.

**Summer Days index**: number of days where daily maximum temperature $TX > T$ (T is  a reference temperature, e.g. 25°C)


As first step, let's connect to the Ophidia Server and import the necessary modules

In [ ]:
from esdm_pav_client import Workflow, Experiment, Task
import sys
from PyOphidia import cube,client
cube.Cube.setclient(read_env=True)

<hr style="height:1px;border-top:1px solid #0000FF" />

As input files, we can use the daily NetCDF files produced by the CMCC-CESM model and related to the tasmax variable for the years 2096-2100. 

In [ ]:
import glob
glob.glob('/home/ophidia/notebooks/tasmax_*.nc')

<hr style="height:1px;border-top:1px solid #0000FF" />

The overall workflow structure is the following:
    
<img src="../imgs/Summer_Days_over_several_year.svg" alt="Summer_Days_over_several_year">

<hr style="height:1px;border-top:1px solid #0000FF" />

To exploit the iterative and parallel interfaces we just need to use the **For Statement**, i.e. the pair of operators **FOR** (with the *parallel* option enabled) and **ENDFOR**.

Let's try to build the workflow step-by-step:

We  need to define the global workflow information followed by the various tasks. Try to complete the various tasks by filling in the missing arguments identified by ```.....```

1. **Create container**
 - We use the *oph_createcontainer* (see http://ophidia.cmcc.it/documentation/users/operators/OPH_CREATECONTAINER.html) to create an empty container in order to organize all the datacubes imported and produced during the workflow execution

In [ ]:
e1 = Experiment(
    name= "Summer Days over several year",
    author= "CMCC",
    abstract= "Summer Days index over several years",
    exec_mode= "sync",
    ncores="1"
)

t1 = e1.newTask(name="Create container",
                type="ophidia",
                operator='oph_createcontainer',
                on_error='skip',
                arguments={'container': 'summer_days',
                           'dim': 'lat|lon|time',
                           'dim_type': 'double|double|double',
                           'hierarchy': 'oph_base|oph_base|oph_time'})

<hr style="height:1px;border-top:1px solid #0000FF" />

The **IMPORT**, **MASK**, **COUNT** and **EXPORT** tasks have to be executed for each of the input years.
So, we use the **FOR** operator to configure the iterative block and, in particular, to set the number N of loops to be executed. 

2. **Start loop**
 - the **values** argument is an ordered list of N labels (pipe-separated years) to be assigned to cycles (i.e., the years from 2096 to 2100) 
 - the **parallel** option is enabled (```yes```)
 - each label value can be read in the inner tasks by using ```@{year}``` as indicated in the **key** argument
 - a dependency from the previous task is required

Each inner task depends on FOR in order to exploit the value of the label associated with the current iteration. 

In [ ]:
t2 = e1.newTask(name="Start loop",
                type="control",
                operator='for',
                arguments={"key": "year", "values": "2096|2097|2098|2099|2100", "parallel": "yes"},
                dependencies={t1:''})

<hr style="height:1px;border-top:1px solid #0000FF" />

3. **Import**
 - the *input* and the *description* parameters in the *oph_importnc* operator are defined in a parametrized way to get the current value of the *year* key for each iteration
 - the task has a simple flow dependency from the "Start loop" task

In [ ]:
t3 = e1.newTask(name="Import",
                type="ophidia",
                operator='oph_importnc',
                arguments={'measure': '$1', 
                           'container':'summer_days', 
                           'import_metadata': 'yes',
                           'imp_dim': 'time', 
                           'imp_concept_level': 'd',
                           'hierarchy': 'oph_base|oph_base|oph_time',
                           'description': 'Max Temp @{year}',
                           'input': '/home/ophidia/notebooks/tasmax_day_CMCC-CESM_rcp85_r1i1p1_@{year}0101-@{year}1231.nc'},
                dependencies={t2:''})

<hr style="height:1px;border-top:1px solid #0000FF" />

4. **Summer Days mask**
 - the *oph_apply* operator (jointly with the *oph_predicate* primitive) is used to identify the summer days: $\{day \mid TX(day) > 298.15\}$ 
 - the task has a data dependency from the *Import* task

In [ ]:
t4 = e1.newTask(name="Summer Days mask",
                type="ophidia",
                operator='oph_apply', 
                arguments={'measure_type': 'auto', 
                           'query': "oph_predicate(measure,'x-298.15','>0','1','0')", 
                           'description': 'Summer days mask @{year}'},
                dependencies={t3:'cube'})

<hr style="height:1px;border-top:1px solid #0000FF" />

5. **Count summer days**
 - count days over the given threshold on yearly basis by using the *oph_reduce2* operator
 - a dependency from *Summer Days mask*

In [ ]:
t5 = e1.newTask(name="Count summer days",
                type="ophidia",
                operator='oph_reduce2', 
                arguments={'operation': 'sum', 
                           'dim': 'time', 
                           'concept_level': 'y',
                           'description': 'Summer Days Count @{year}'},
                dependencies={t4:'cube'})

<hr style="height:1px;border-top:1px solid #0000FF" />

6. **Export**
 - each Summer Days index is exported to a NetCDF file by using the *oph_exportnc2* operator 
 - a dependency from *Count summer days*

In [ ]:
t6 = e1.newTask(name="Export",
                type="ophidia",
                operator='oph_exportnc2', 
                arguments={'output': '/home/ophidia/notebooks/summer_days_@{year}.nc'},
                dependencies={t5:'cube'})

<hr style="height:1px;border-top:1px solid #0000FF" />

The iterative block ends by means of the **ENDFOR** operator.

7. **End loop**
 - *endfor* operator
 - a dependency from **Export** task

In [ ]:
t7 = e1.newTask(name="End loop",
                type="control",
                operator='endfor',
                arguments={},
                dependencies={t6:'cube'})

<hr style="height:1px;border-top:1px solid #0000FF" />

Check the workflow structure

In [ ]:
e1.check(visual=True)

<hr style="height:1px;border-top:1px solid #0000FF" />

Submit the workflow and monitor its execution graph

In [ ]:
w1 = Workflow(e1)
w1.submit("tasmax")

In [ ]:
w1.monitor(frequency=1, iterative=True, visual_mode=True)

<hr style="height:1px;border-top:1px solid #0000FF" />

Run the following command. You should see a netCDF file for each year:
- summer_days_2096.nc
- summer_days_2097.nc
- summer_days_2098.nc
- summer_days_2099.nc
- summer_days_2100.nc

In [ ]:
import glob
glob.glob('/home/ophidia/notebooks/summer_*.nc')

List the datacubes produced

In [ ]:
cube.Cube.list(level=2)

Let's plot a map for each year by considering the PIDs associated to the **Summer Days Count @{year}** datacubes. Starting from the output of the previous command, we can define an array of PIDs (sorted by year) and use it in the code below.

In [ ]:
pids = ["http://127.0.0.1/ophidia/.../...","http://127.0.0.1/ophidia/.../...",
       "http://127.0.0.1/ophidia/.../...","http://127.0.0.1/ophidia/.../...","http://127.0.0.1/ophidia/.../..."]

In [ ]:
%matplotlib inline
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
from cartopy.mpl.geoaxes import GeoAxes
from cartopy.util import add_cyclic_point
import numpy as np
import warnings
warnings.filterwarnings("ignore")

titles = ["Summer Days (year 2096)","Summer Days (year 2097)","Summer Days (year 2098)","Summer Days (year 2099)","Summer Days (year 2100)"]

#Add Geo axes to the figure with the specified projection (PlateCarree)
projection = ccrs.PlateCarree()
fig, axes = plt.subplots(nrows=3,ncols=2, figsize=(18, 12), subplot_kw={'projection': projection})
fig.delaxes(axes[2][1])

for v,ax in enumerate(axes.flat):
    
    try:
        datacube = cube.Cube(pid=pids[v])
        data = datacube.export_array(show_time='yes')
        lat = data['dimension'][0]['values'][ : ]
        lon = data['dimension'][1]['values'][ : ]
        var = data['measure'][0]['values'][ : ]
        var = np.reshape(var, (len(lat), len(lon)))
    except:
         continue
            
    #Draw coastline and gridlines
    ax.coastlines()

    gl = ax.gridlines(crs=projection, draw_labels=True, linewidth=1, color='black', alpha=0.9, linestyle=':')
    gl.xlabels_top = False
    gl.ylabels_right = False

    #Wraparound points in longitude
    var_cyclic, lon_cyclic = add_cyclic_point(var, coord=np.asarray(lon))
    x, y = np.meshgrid(lon_cyclic,lat)

    #Define color levels for color bar
    levStep = (np.nanmax(var)-np.nanmin(var))/20
    clevs = np.arange(np.nanmin(var),np.nanmax(var)+levStep,levStep)

    #Set filled contour plot
    cnplot = ax.contourf(x, y, var_cyclic, clevs, transform=projection,cmap=plt.cm.Oranges)
    ax.set_title(titles[v])
    plt.colorbar(cnplot,ax=ax)

    ax.set_aspect('auto', adjustable=None)

plt.show()

Clear the cube space before running the other workflows

In [ ]:
cube.Cube.deletecontainer(container="summer_days",force='yes')

You can now move to the last hands-on notebook [**3-Selection_interface**](3-Selection_interface.ipynb).